In [1]:
import torch
import torch.nn as nn
from torchsummary import summary

import torchvision

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

/opt/homebrew/Caskroom/miniconda/base/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [3]:
class AlexNet(nn.Module):
    def __init__(self, in_channels: int = 3, out_channels: int = 256, out_features: int = 1000):
        super(AlexNet, self).__init__()
        
        # 3x277x277 -> 96x27x27
        self.layer1 = nn.Sequential(
            # 3x227x227 -> 96x55x55
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=96,
                kernel_size=11,
                stride=4,
                padding=0
            ),
            nn.ReLU(inplace=True),
            # 96x55x55 -> 96x27x27
            nn.MaxPool2d(
                kernel_size=3,
                stride=2,
                padding=0
            )
        )
        # 96x27x27 -> 256x13x13
        self.layer2 = nn.Sequential(
            # 96x27x27 -> 256x27x27
            nn.Conv2d(
                in_channels=96,
                out_channels=256,
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(inplace=True),
            # 256x27x27 -> 256x13x13
            nn.MaxPool2d(
                kernel_size=3,
                stride=2,
                padding=0
            )
        )
        # 256x13x13 -> 384x13x13
        self.layer3 = nn.Sequential(
            # 256x13x13 -> 384x13x13
            nn.Conv2d(
                in_channels=256,
                out_channels=384,
                kernel_size=3,
                stride=2,
                padding=0
            ),
            nn.ReLU(inplace=True)
        )
        # 384x13x13 -> 384x13x13
        self.layer4 = nn.Sequential(
            # 384x13x13 -> 384x13x13
            nn.Conv2d(
                in_channels=384,
                out_channels=384,
                kernel_size=3,
                stride=2,
                padding=0
            ),
            nn.ReLU(inplace=True)
        )
        # 384x13x13 -> 256x6x6
        self.layer5 = nn.Sequential(
            # 384x13x13 -> 256x13x13
            nn.Conv2d(
                in_channels=384,
                out_channels=out_channels,
                kernel_size=3,
                stride=2,
                padding=0
            ),
            nn.ReLU(),
            # 256x13x13 -> 256x6x6
            nn.MaxPool2d(
                kernel_size=3,
                stride=2,
                padding=0
            )
        )
        # 256*6*6 -> 4096
        self.layer6 = nn.Sequential(
            nn.Dropout(),
            # 256*6*6 -> 4096
            nn.Linear(
                in_features=256*6*6,
                out_features=4096
            ),
            nn.ReLU(inplace=True)
        )
        # 4096 -> 4096
        self.layer7 = nn.Sequential(
            nn.Dropout(),
            nn.Linear(
                in_features=4096,
                out_features=4096
            ),
            nn.ReLU(inplace=True)
        )
        # 4096 -> 1000
        self.layer8 = nn.Sequential(
            nn.Linear(
                in_features=4096,
                out_features=out_features
            ),
            nn.Softmax(
                dim=1
            )
        )
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)

        x = torch.view(-1, 256*6*6)

        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        return x

In [4]:
model = AlexNet().to(device)

summary(model, input_size=(3, 277, 277))

RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small